In [4]:
%pip install backoff
%pip install ratelimit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached ratelimit-2.2.1.tar.gz (5.3 kB)
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5908 sha256=a42b8c60db248513f9b9cdd1cf8e2a3266ba1879693b800baa123f6546fc4339
  Stored in directory: /home/kfeng18/.cache/pip/wheels/14/1e/97/126009a0884bdf7e26436cace73d9a4f4596dada4fdc4950ce
Successfully built ratelimit
Note: you may need to restart the kernel to use updated packages.


In [1]:
from backoff import on_exception, expo
from collections import defaultdict
import numpy as np
import pandas as pd
import requests
from ratelimit import limits, sleep_and_retry

In [2]:
input_df = pd.read_csv(
    '../data/conmed_example_data_with_best_rxcuid.csv',
    index_col=0).fillna(np.nan).replace([np.nan], [None])

In [3]:
input_df['Best RxNorm Id'] = [
    str(int(rxcui)) if rxcui else None for rxcui in input_df['Best RxNorm Id'] ]

In [4]:
base_api_url = 'https://rxnav.nlm.nih.gov/REST'

@sleep_and_retry
@on_exception(expo, ConnectionError, max_tries=10)
@limits(calls=20, period=2)
def call_class_api(cui):
    api_url = '{}/rxclass/class/byRxcui.json?rxcui={}'.format(
        base_api_url, rxcui)
    response = requests.get(api_url)
    if response.status_code == 403:
        raise ConnectionError('API response: {}'.format(response.status_code))
    response_json = response.json()
    return response_json

@sleep_and_retry
@on_exception(expo, ConnectionError, max_tries=10)
@limits(calls=20, period=2)
def call_class_graph_api(clz_id, src):
    api_url = '{}/rxclass/classGraph.json?classId={}&source={}'.format(
        base_api_url, clz_id, src)
    response = requests.get(api_url)
    if response.status_code == 403:
        raise ConnectionError('API response: {}'.format(response.status_code))
    response_json = response.json()
    return response_json

def get_new_relationship(rxcui):
    all_clz = call_class_api(rxcui)
    if all_clz:
        rx_class_info = all_clz['rxclassDrugInfoList']['rxclassDrugInfo']

        src_clz_relationship = dict()
        for concept in rx_class_info:
            if concept['rxclassMinConceptItem']['classType'] not in src_clz_relationship:
                src_clz_relationship[concept['rxclassMinConceptItem']['classType']] = dict()
            if concept['rela'] not in src_clz_relationship[concept['rxclassMinConceptItem']['classType']]:
                src_clz_relationship[concept['rxclassMinConceptItem']['classType']][concept['rela']] = set()
            src_clz_relationship[concept['rxclassMinConceptItem']['classType']][concept['rela']].add(
                concept['rxclassMinConceptItem']['classId'])

        src_relationship = dict()
        for src, rela_ids in src_clz_relationship.items():
            if src not in src_relationship:
                src_relationship[src] = dict()
            for rela, clz_ids in rela_ids.items():
                for clz_id in clz_ids:
                    response_json = call_class_graph_api(clz_id, src)
                    if response_json['rxclassGraph']:
                        if rela not in src_relationship[src]:
                            src_relationship[src][rela] = set()
                        src_relationship[src][rela].add(
                            response_json['rxclassGraph']['rxclassMinConceptItem'][0]['className'])
                        if len(response_json['rxclassGraph']['rxclassMinConceptItem']) > 1:
                            for concept in response_json['rxclassGraph']['rxclassMinConceptItem'][1:]:
                                concept_name = concept['className']
                                src_relationship[src][rela].add(concept_name)

        src_relationship_lists = dict()
        for src, relas in src_relationship.items():
            for rela, concept in relas.items():
                if not rela:
                    comb_src = src
                else:
                    comb_src = '_'.join([src, rela])
                src_relationship_lists[comb_src] = ';'.join(concept)

        new_relationship = pd.DataFrame(src_relationship_lists, index=[0])
    else:
        new_relationship = pd.DataFrame()
    new_relationship['Best RxNorm Id'] = rxcui
    return new_relationship

In [5]:
relationship_df = pd.DataFrame()  

In [6]:
for rxcui in set(input_df['Best RxNorm Id']):
    if rxcui:
        print('Working on {}'.format(rxcui))
        new_relationship = get_new_relationship(rxcui)
        relationship_df = pd.concat([relationship_df, new_relationship], ignore_index=True)

Working on 756905
Working on 18993
Working on 2598
Working on 227682
Working on 203029
Working on 91818
Working on 25025
Working on 226084
Working on 20489
Working on 1895
Working on 18631
Working on 25480
Working on 1664316
Working on 301543
Working on 1160453
Working on 26225
Working on 1114038
Working on 4821
Working on 153010
Working on 3640
Working on 220224
Working on 1191
Working on 1876402
Working on 11253
Working on 41126
Working on 1151
Working on 214559
Working on 203302
Working on 1234482
Working on 161
Working on 203345
Working on 11251
Working on 1234386
Working on 227783
Working on 33738
Working on 28889
Working on 801177
Working on 19831
Working on 2200645
Working on 495430
Working on 4301
Working on 209468
Working on 2556
Working on 218402
Working on 11416
Working on 10324
Working on 2479830
Working on 1588
Working on 11248
Working on 10582
Working on 1117392
Working on 6809
Working on 5032
Working on 83373
Working on 29046
Working on 1790650
Working on 435
Working on 

In [7]:
relationship_df

DISPOS_isa_disposition  \
0   Medicinal product categorized by disposition;O...   
1                                                 NaN   
2                                                 NaN   
3   Medicinal product categorized by disposition;I...   
4                                                 NaN   
..                                                ...   
71  Medicinal product categorized by disposition;E...   
72  Serotonin 5-HT1 receptor agonist-containing pr...   
73  Medicinal product categorized by disposition;I...   
74                                                NaN   
75                                                NaN   

                                 STRUCT_isa_structure  \
0   Alkaloid-containing product;Ether structure-co...   
1   Nitrogen and/or nitrogen compound-containing p...   
2   Benzodiazepine-containing product;Medicinal pr...   
3   Medicinal product categorized by structure;Glu...   
4                                                 NaN   
..                                                ...   
71  Organic acid and/or organic acid derivative-co...   
72  Sulfur and/or sulfur compound-containing produ...   
73  Medicinal product categorized by structure;Glu...   
74                                                NaN   
75  Sulfur and/or sulfur compound-containing produ...   

                                               MESHPA  \
0   Therapeutic Uses;Analgesics, Opioid;Sensory Sy...   
1   Therapeutic Uses;Central Nervous System Agents...   
2   GABA Agents;Therapeutic Uses;GABA Modulators;N...   
3   Autonomic Agents;Therapeutic Uses;Antineoplast...   
4   Therapeutic Uses;Antimanic Agents;Cardiovascul...   
..                                                ...   
71  Therapeutic Uses;Anti-Inflammatory Agents;Anti...   
72  Therapeutic Uses;Serotonin 5-HT1 Receptor Agon...   
73  Antineoplastic Agents;Therapeutic Uses;Anti-In...   
74                                                NaN   
75  MeSH Pharmacologic Actions (MESHPA);Antithyroi...   

                                      DISEASE_ci_with  \
0   Lung Diseases;Respiratory Tract Diseases;Repro...   
1   Chemically-Induced Disorders;Disease;Drug-Rela...   
2   Chemically-Induced Disorders;Glaucoma, Angle-C...   
3   Virus Diseases;Mycoses;Chemically-Induced Diso...   
4   Chemically-Induced Disorders;Pregnancy;Life Ph...   
..                                                ...   
71  Lung Diseases;Respiratory Tract Diseases;Pregn...   
72  Chest Pain;Migraine Disorders;Ischemia;Stroke;...   
73  Virus Diseases;Mycoses;Chemically-Induced Diso...   
74                                                NaN   
75  Chemically-Induced Disorders;Pregnancy;Life Ph...   

                                    DISEASE_may_treat  \
0   Lung Diseases;Sinusitis;Bordetella Infections;...   
1   Diseases, Life Phases, Behavior Mechanisms and...   
2   Speech Disorders;Epilepsy;Lennox Gastaut Syndr...   
3   Inflammatory Bowel Diseases;Purpura, Thrombocy...   
4   Epilepsy;Epilepsies, Partial;Facial Neuralgia;...   
..                                                ...   
71  Spondylarthritis;Osteoarthritis;Arthritis;Conn...   
72  Trigeminal Autonomic Cephalalgias;Diseases, Li...   
73  Inflammatory Bowel Diseases;Purpura, Thrombocy...   
74                                                NaN   
75  Endocrine System Diseases;Thyroid Crisis;Thyro...   

                                  CHEM_has_ingredient  \
0   Heterocyclic Compounds, 4 or More Rings;Phenol...   
1   Substances;Alkanes;Butanes;Amines;Organic Chem...   
2   Heterocyclic Compounds, Fused-Ring;Heterocycli...   
3   Pregnadienetriols;Steroids, Fluorinated;Polycy...   
4   Heterocyclic Compounds;Heterocyclic Compounds,...   
..                                                ...   
71  Ibuprofen;Amino Acids;Carboxylic Acids;Lysine;...   
72  Heterocyclic Compounds, Fused-Ring;Heterocycli...   
73  Polycyclic Compounds;Pregnadienes;Substances;S...   
74  mRNA Vaccines;Vaccines;Complex M

In [8]:
output_df = input_df.merge(relationship_df, how='outer', on='Best RxNorm Id')

In [9]:
output_df

ID                    Event          Item Dictionary   Verbatim Term  \
0     1  Concomitant Medications  Generic Name     RxNorm    escitalopram   
1    30  Concomitant Medications  Generic Name     RxNorm    escitalopram   
2     1  Concomitant Medications  Generic Name     RxNorm          mirena   
3    16  Concomitant Medications  Generic Name     RxNorm          Mirena   
4     2  Concomitant Medications  Generic Name     RxNorm   norethindrone   
..   ..                      ...           ...        ...             ...   
150  43  Concomitant Medications  Generic Name     RxNorm     Guaifenesin   
151  43  Concomitant Medications  Generic Name     RxNorm         Flonase   
152  43  Concomitant Medications  Generic Name     RxNorm     Ondansetron   
153  43  Concomitant Medications  Generic Name     RxNorm  ofloxacin otic   
154  45  Concomitant Medications  Generic Name     RxNorm            Zinc   

                  Preferred Term      RXCUI                    Drug Name  \
0             Escitalopram 20 MG   353383.0           Escitalopram 20 MG   
1                   Escitalopram   321988.0                 Escitalopram   
2                         Mirena   152061.0                       Mirena   
3    Mirena Drug Implant Product  1181044.0  Mirena Drug Implant Product   
4                           None       None                         None   
..                           ...        ...                          ...   
150             Guaifenesin Pill  1159705.0             Guaifenesin Pill   
151        Flonase Nasal Product  1801248.0        Flonase Nasal Product   
152                  Ondansetron    26225.0                  Ondansetron   
153      Ofloxacin Otic Solution   377450.0      Ofloxacin Otic Solution   
154                         Zinc    11416.0                         Zinc   

    Normalized RxNorm Id Approximate RxNorm Id  ...  \
0               321988.0              321988.0  ...   
1               321988.0              321988.0  ...   
2               152061.0              152061.0  ...   
3               152061.0              152061.0  ...   
4                 7514.0                7514.0  ...   
..                   ...                   ...  ...   
150               5032.0                5032.0  ...   
151              83373.0               83373.0  ...   
152              26225.0               26225.0  ...   
153            1160453.0             1160453.0  ...   
154              11416.0               11416.0  ...   

                                        VA_has_VAClass  \
0    CENTRAL NERVOUS SYSTEM MEDICATIONS;ANTIDEPRESS...   
1    CENTRAL NERVOUS SYSTEM MEDICATIONS;ANTIDEPRESS...   
2    CONTRACEPTIVES,SYSTEMIC;HORMONES/SYNTHETICS/MO...   
3    CONTRACEPTIVES,SYSTEMIC;HORMONES/SYNTHETICS/MO...   
4    CONTRACEPTIVES,SYSTEMIC;ANTINEOPLASTICS;VA CLA...   
..                                                 ...   
150  DECONGESTANT/EXPECTORANT;VA CLASSES;RESPIRATOR...   
151  ANTI-INFLAMMATORIES,NASAL;NASAL AND THROAT AGE...   
152  GASTROINTESTINAL MEDICATIONS;ANTIEMETICS;VA CL...   
153  OTIC AGENTS;ANTI-INFECTIVE,TOPICAL OTIC;VA CLA...   
154                                                NaN   

                               VA_has_VAClass_extended  \
0    CENTRAL NERVOUS SYSTEM MEDICATIONS;ANTIDEPRESS...   
1    CENTRAL NERVOUS SYSTEM MEDICATIONS;ANTIDEPRESS...   
2    CONTRACEPTIVES,SYSTEMIC;HORMONES/SYNTHETICS/MO...   
3    CONTRACEPTIVES,SYSTEMIC;HORMONES/SYNTHETICS/MO...   
4    CONTRACEPTIVES,SYSTEMIC;ANTINEOPLASTICS;VA CLA...   
..                                                 ...   
150  DECONGESTANT/EXPECTORANT;VA CLASSES;RESPIRATOR...   
151  ANTI-INFLAMMATORIES,NASAL;NASAL AND THROAT AGE...   
152  GASTROINTESTINAL MEDICATIONS;ANTIEMETICS;VA CL...   
153  OTIC AGENTS;ANTI-INFECTIVE,TOPICAL OTIC;VA CLA...   
154                                                NaN   

                                SCHEDULE_has_schedule  \
0                                                 NaN  

In [10]:
output_df.to_csv('../data/conmed_example_data_with_hierarchy.csv')